In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Tải stopwords tiếng Việt nếu chưa có
nltk.download('stopwords')
vietnamese_stopwords = set(stopwords.words('english'))  # Chưa có stopwords tiếng Việt sẵn

# 📌 Bước 1: Đọc dữ liệu
file_path = "/content/drive/MyDrive/BTL_DataMining/Data_BTL/shuffled_data_vnexpress_titles.csv"  # Đổi đường dẫn nếu cần
df = pd.read_csv(file_path)

# 📌 Bước 2: Tiền xử lý dữ liệu
def preprocess_text(text):
    text = text.lower()  # Chuyển thành chữ thường
    text = re.sub(r'\d+', '', text)  # Loại bỏ số
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ ký tự đặc biệt
    text = ' '.join([word for word in text.split() if word not in vietnamese_stopwords])  # Loại bỏ stopwords
    return text

df.dropna(inplace=True)  # Xóa dòng có giá trị trống
df['Title'] = df['Title'].astype(str).apply(preprocess_text)  # Áp dụng xử lý

# 📌 Bước 3: Chia dữ liệu thành tập train và test
X = df['Title']
y = df['Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 Bước 4: Biểu diễn văn bản thành vector TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Chỉ giữ 5000 từ quan trọng nhất
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 📌 Bước 5: Huấn luyện mô hình SVM
svm_model = SVC(kernel='linear')  # Dùng kernel tuyến tính cho bài toán phân loại văn bản
svm_model.fit(X_train_tfidf, y_train)

# 📌 Bước 6: Dự đoán trên tập kiểm tra
y_pred = svm_model.predict(X_test_tfidf)

# 📌 Bước 7: Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f"🎯 Accuracy: {accuracy:.4f}")
print("📊 Classification Report:\n", classification_report(y_test, y_pred))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


🎯 Accuracy: 0.7241
📊 Classification Report:
               precision    recall  f1-score   support

   Công nghệ       0.80      0.80      0.80        10
    Giáo dục       1.00      0.78      0.88         9
    Giải trí       0.78      0.58      0.67        12
  Kinh doanh       0.43      0.67      0.52         9
    Sức khỏe       0.88      0.64      0.74        11
    Thể thao       0.70      1.00      0.82         7

    accuracy                           0.72        58
   macro avg       0.76      0.74      0.74        58
weighted avg       0.77      0.72      0.73        58

